In [14]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import fnmatch
import csv
import pickle
import warnings
import torch
import scipy
from scipy import optimize
from scipy.optimize import leastsq
from io import StringIO
from scipy.signal import savgol_filter
from datetime import datetime

now = datetime.now()
d_string = now.strftime("%d.%m.%Y")



In [103]:
time = 10
energy = 200
nbins = 60
training_epochs = 50000
Nrep = 10
minval = -1
maxval = +1

path_1 = r'Data/Results/%(now)s/' % {"now": d_string} # use your path
path_2 = r'xyp_%(energy)s_%(time)sms_*' % {"energy": energy, "time": time}
all_files = glob.glob(path_1 + path_2)

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [104]:
pseudo_x = frame.iloc[:,0]
pseudo_y = frame.iloc[:,1]
pseudo_pred = frame.iloc[:,2]

In [105]:
%run Functions.ipynb


In [106]:
pseudo_mean, pseudo_std, pseudo_meanvector, pseudo_stdvector = get_mean_pseudodata(pseudo_x, pseudo_pred)
pseudo_xvector = np.linspace(-1, 1, nbins)

## Find the residuals

In [100]:
def residuals(prediction, y, std):
    res = np.divide((prediction - y), std)
    return res

In [101]:
chi = np.loadtxt('Data/Results/%(date)s/aaa chidistribution_%(energy)s_%(time)sms_e%(epoch)s_Nrep=%(i)s.csv' % {"date": d_string, "energy":energy, "time": time, "epoch": training_epochs, "i": Nrep })

best_value = min(chi)
best_try = (np.where(chi == best_value))
best_try = int(np.rint(best_try))
N_train = int(np.divide(len(pseudo_x), Nrep))

best_pred = pseudo_pred[best_try*N_train :(best_try+1)*N_train]
best_y = pseudo_y[best_try*N_train :(best_try+1)*N_train]
